### ✅ Problem: File Compression Tool
- You're building a simple file compression utility. Internally, there are different compression algorithms: zip, tar, and gzip.

In [3]:
from abc import ABC, abstractmethod
class ICompressor(ABC):
    @abstractmethod
    def compress(self, data):
        """
        Compress the given data.

        :param data: The data to compress.
        :return: The compressed data.
        """
        raise NotImplementedError("Subclasses should implement this method.")
    
    @abstractmethod
    def decompress(self, data):
        """
        Decompress the given data.

        :param data: The data to decompress.
        :return: The decompressed data.
        """
        raise NotImplementedError("Subclasses should implement this method.")
    



class GzipCompressor(ICompressor):
    def compress(self, data):
        import gzip
        return gzip.compress(data.encode('utf-8'))

    def decompress(self, data):
        import gzip
        return gzip.decompress(data).decode('utf-8')

class ZipCompressor(ICompressor):
    def compress(self, data):
        import zipfile
        import io
        buffer = io.BytesIO()
        with zipfile.ZipFile(buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.writestr('data.txt', data.encode('utf-8'))
        return buffer.getvalue()

    def decompress(self, data):
        import zipfile
        import io
        buffer = io.BytesIO(data)
        with zipfile.ZipFile(buffer, 'r') as zip_file:
            return zip_file.read('data.txt').decode('utf-8')
        
    
class TarCompressor(ICompressor):
    def compress(self, data):
        import tarfile
        import io
        buffer = io.BytesIO()
        with tarfile.open(fileobj=buffer, mode='w:gz') as tar_file:
            info = tarfile.TarInfo(name='data.txt')
            info.size = len(data.encode('utf-8'))
            tar_file.addfile(tarinfo=info, fileobj=io.BytesIO(data.encode('utf-8')))
        return buffer.getvalue()

    def decompress(self, data):
        import tarfile
        import io
        buffer = io.BytesIO(data)
        with tarfile.open(fileobj=buffer, mode='r:gz') as tar_file:
            return tar_file.extractfile('data.txt').read().decode('utf-8')
        
        
class Compressor(ICompressor):
    def __init__(self):
        self._file_operation_fun = {}

    def compress(self, data,compressor_name: str = None):
        file_operation_fun = self._file_operation_fun.get(compressor_name)
        if file_operation_fun is None:
            raise ValueError(f"Compressor '{compressor_name}' not found.")
        
        return file_operation_fun.compress(data)

    def decompress(self, data,decompressor_name: str = None):
        file_operation_fun = self._file_operation_fun.get(decompressor_name)
        if file_operation_fun is None:
            raise ValueError(f"Compressor '{decompressor_name}' not found.")
        
        return file_operation_fun.decompress(data)
        
    def add_compressor(self, file_operation_fun: ICompressor,compressor_name: str = None):
        self._file_operation_fun[compressor_name] = file_operation_fun
        return self

file_object = Compressor()
file_object.add_compressor(GzipCompressor(),compressor_name='gzip')\
    .add_compressor(ZipCompressor(),compressor_name='zip')\
    .add_compressor(TarCompressor(),compressor_name='tar')
    
file_object.compress('hello world',compressor_name='gzip')


b'\x1f\x8b\x08\x00\xf7Z\x1bh\x02\xff\xcbH\xcd\xc9\xc9W(\xcf/\xcaI\x01\x00\x85\x11J\r\x0b\x00\x00\x00'